In [25]:
import sys
from time import sleep
sys.path.append('..')
from pprint import pprint
import time
from RaAuto.Pmo import OpenPMO, HandleInternalMail
from RaAuto.RaAuto import *
from RaAuto.RaTime import add_minutes
import pyautogui as pag
from PIL import Image, ImageDraw

In [26]:
class ImgsEnum:
    def __init__(self, imagesPath):
        self.TenMinuteMark = os.path.join(imagesPath, "Schedule", "TenMinuteMark.png")
        self.AndraMallTillfalligt = os.path.join(imagesPath, "Schedule", "AndraMallTillfalligt.png")
        self.IdagButton = os.path.join(imagesPath, "IdagButton.png")
        self.ResurserLabel = os.path.join(imagesPath, "Schedule", "ResurserLabel.png")
        self.StaffMemberCheckboxChecked = os.path.join(imagesPath, "Schedule", "StaffMemberCheckboxChecked.png")
        self.StaffMemberCheckboxCheckedOnUnselectedRow = os.path.join(imagesPath, "Schedule", "StaffMemberCheckboxCheckedOnUnselectedRow.png")
        self.NyBokningTitle = os.path.join(imagesPath, "NyBokningTitle.png")
        self.WarningSymbolDoubleBookingLarge = os.path.join(imagesPath, "WarningSymbolDoubleBookingLarge.png")
        self.AvbrytMenuButton = os.path.join(imagesPath, "AvbrytMenuButton.png")
        self.VardgarantiCheckboxChecked = os.path.join(imagesPath, "VardgarantiCheckboxChecked.png")
        self.EightOClockMark = os.path.join(imagesPath, "Schedule", "EightOClockMark.png")
        self.TwelveOClockMark = os.path.join(imagesPath, "Schedule", "TwelveOClockMark.png")
        self.SixteentOClockMark = os.path.join(imagesPath, "Schedule", "SixteentOClockMark.png")


IMG = ImgsEnum(os.path.join("..", "Images"))

In [27]:
def GoToUserSelectionArea():
    HandleInternalMail()
    pos = waitForElement(IMG.ResurserLabel)
    if (pos):
        x = pos.x + 50
        y = pos.y + 100
        return Pos(x, y)
    return False


In [28]:
def ScrollUp():
    sleep(0.2)
    pag.scroll(500)
    sleep(0.2)
    pag.scroll(500)
    sleep(0.2)

In [29]:
def DeselectAllUsers(repeats=5):
    pos = GoToUserSelectionArea()

    if not pos:
        return False
    
    pag.moveTo(pos.x, pos.y)
    ScrollUp()

    for i in range(repeats):
        for j in range(3):
            waitAndClickElement(IMG.StaffMemberCheckboxChecked, beforeStart=0.1, limit=0.1, waitTime=0.1, warn=False)
        pag.moveTo(pos.x, pos.y)
        pag.scroll(-250)
        sleep(0.2)

    if HandleInternalMail():
        pos = DeselectAllUsers(repeats=5)
        
    return pos


# pos = OpenPMO()
# if pos:
#     DeselectAllUsers()

In [30]:
def DeselectSingleUser():
    return waitAndClickElement(IMG.StaffMemberCheckboxChecked, beforeStart=0.1, limit=0.1, waitTime=0.1, warn=False)

In [31]:
def CheckIfUsesIsSelectedCorrectly(userName):
    pos = waitForElement(IMG.IdagButton)
    if pos:
        text = capture_and_ocr(pos.x, pos.y + 13, width=500)
        print(f'Text: "{text}"')
        if userName in text:
            print("User correctly selected")
            return pos
        else:
            print("USER NOT CORRECTLY SELECTED!")
            return False

In [32]:
def SetUser(userName):
    # DeselectSingleUser()

    pos = GoToUserSelectionArea()

    if not pos:
        return False
    
    pag.moveTo(pos.x, pos.y)
    # ScrollUp()
    pag.leftClick()

    # words = userName.split(" ")
    waitAndType(userName)
    waitAndPress("space")
    pos = CheckIfUsesIsSelectedCorrectly(userName)
    print(f"User was {'selected correctly' if pos else 'not selected correctly'}")
    if pos:
        return pos
    else:
        if " " in userName:
            waitAndClickElement(IMG.StaffMemberCheckboxCheckedOnUnselectedRow, relX=9)
    
    pos = CheckIfUsesIsSelectedCorrectly(userName)
    if pos:
        return pos
    else:
        raise RuntimeError("Could not select user {userName} correctly")
    

# pos = OpenPMO()
# if pos:
#     DeselectAllUsers()
#     SetUser("Linda Y")


In [33]:
def SetDate(date):
    pos = waitAndClickElement(IMG.IdagButton, relX=-60)
    if pos:
        waitAndPress("ctrl", "a")
        waitAndType(date)
        if HandleInternalMail():
            pos = SetDate(date)
        else:
            waitAndPress("enter")
    return pos
    

# pos = OpenPMO()
# if pos:
#     SetDate("2024-06-03")

In [34]:
# negativePreviousCount is because the dialogue may already have a count set higher than 1
def SetTimeSlots(typeStr, startTime, count, negativePreviousCount=0):
    pos = waitAndClickElement(IMG.TenMinuteMark, relX=15, button="right")
    if pos:
        pos = waitAndPress("m")
    if pos:
        pos = waitForElement(IMG.AndraMallTillfalligt)
    if pos:
        waitAndType(typeStr)
        waitAndPress("tab")
        waitAndType(startTime)
        sleep(0.05)
        waitAndPress("tab")
        waitAndPress("tab")
        if (negativePreviousCount > 0):
            sleep(0.4)
            for i in range(negativePreviousCount):
                waitAndPress("down", beforeStart=0.07)
        sleep(0.4)
        for i in range (1, count):
            waitAndPress("up", beforeStart=0.07)
        sleep(0.05)
        if HandleInternalMail():
            waitAndPress("alt", "o")
            pos = SetTimeSlots(typeStr, startTime, count, negativePreviousCount)
        else:
            waitAndPress("alt", "o")
            pos = waitForElementDisappear(IMG.AndraMallTillfalligt)
            sleep(0.1)

    return pos


# pos = OpenPMO()
# if pos:
#     SetTimeSlots("ad", "08:00", 30, 30)

In [35]:
def AddBooking(comment, type, startTime, minutes):

    pos = waitAndClickElement(IMG.TenMinuteMark, relX=12)
    waitAndPress("alt", "n")
    pos = waitForElement(IMG.NyBokningTitle)

    if pos:
        tempPos = waitForElement(IMG.WarningSymbolDoubleBookingLarge, limit=1, waitTime=0.3, warn=False)
        if tempPos:
            waitAndPress("alt", "o")

        for i in range(9):
            waitAndPress("tab", beforeStart=0.025)
        waitAndType(type)
        waitAndPress("tab")
        waitAndPress("tab")
        waitAndType(startTime)
        waitAndPress("tab")
        waitAndType(str(minutes))
        waitAndPress("tab")
        waitAndPress("tab")
        if HandleInternalMail():
            waitAndClickElement(IMG.AvbrytMenuButton, limit=1)
            waitForElementDisappear(IMG.NyBokningTitle)
            sleep(0.5)
            pos = AddBooking(comment, type, startTime, minutes)
        else:
            waitAndType(comment)
            waitAndClickElement(IMG.VardgarantiCheckboxChecked, relX=-7, limit=0.2, conf=0.95, warn=False)

            if HandleInternalMail():
                waitAndClickElement(IMG.AvbrytMenuButton, limit=1)
                waitForElementDisappear(IMG.NyBokningTitle)
                sleep(0.5)
                pos = AddBooking(comment, type, startTime, minutes)
            else:
                waitAndPress("alt", "s")
                tempPos = waitForElement(IMG.WarningSymbolDoubleBookingLarge, limit=1.5, waitTime=0.5, warn=False)
                if tempPos:
                    waitAndPress("enter")
            waitForElementDisappear(IMG.NyBokningTitle)
            sleep(0.5)

    return pos

# pos = OpenPMO()
# if pos:
#     AddBooking("Special2", "ad", "10:00", 40)



# Import and filters

In [36]:
from datetime import datetime
import json

In [37]:
with open('Input/schedule.json') as file:
    scheduleData = json.load(file)

staff_dict = scheduleData["staffMemberDict"]
room_dict = scheduleData["roomDict"]
staff_schedules = scheduleData["schedule"]

In [38]:
room_dict

{'1': 'ST rum 1',
 '2': 'ST rum 2',
 '3': 'LW rum',
 '4': 'RA rum',
 '5': 'MS rum',
 '6': 'Lokalen',
 '7': 'Hemifrån',
 '8': 'Annat rum',
 '9': 'Ej på vc'}

In [39]:
activityDict ={
    "ns": "<",
    "ad": "ad",
    "em": "ak",
    "pl": "mo",
    "ph": "tel",
    "br": "ra",
    "me": "ex",
    "sp": "sp",
    "mi": "öv",
    "ch": "läkartid b",
    "tr": "tea",
    "wl": "be",
}

In [40]:
staff_dict

{'1': 'Rasmus',
 '2': 'Linn',
 '3': 'Maria MC',
 '4': 'Maria K',
 '5': 'Dhay',
 '6': 'Michael',
 '7': 'Julijana',
 '8': 'Anna',
 '9': 'Isac'}

## FILTER

In [41]:
#staff_schedules[0]["personDays"][0]

# Filters:
start_date = "2024-11-18" # "2024-08-12"
end_date = "2024-11-29" # "2024-08-23"
staff_members = None # [1, 2, 3, 4, 5, 6, 8, 9, 10]

print(len(staff_schedules))
filtered_staff_schedules = staff_schedules
if staff_members:
    filtered_staff_schedules = [schedule for schedule in staff_schedules if schedule["staffPk"] in staff_members]
print(len(filtered_staff_schedules))

8
8


In [42]:
if isinstance(start_date, str):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
if isinstance(end_date, str):
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

print(len(filtered_staff_schedules[0]["personDays"]))
if start_date and end_date:
    filtered_staff_schedules = [
        {
            "staffPk": schedule["staffPk"],
            "personDays": [
                person_day for person_day in schedule["personDays"]
                if start_date <= datetime.strptime(person_day["date"], "%Y-%m-%d") <= end_date
            ]
        }
        for schedule in filtered_staff_schedules
    ]

print(len(filtered_staff_schedules[0]["personDays"]))

28
12


In [43]:
# example_chunk = staff_schedules[0]["personDays"][0]["schedule_chunk"]
# example_chunk

In [44]:
def SetScheduleChunk(chunk):
    start_time = chunk["start_time"][:5]
    time_resolution = chunk["time_resolution"]
    time_slots = chunk["time_slots"]

    index = 0
    prev_count = 30

    while index < len(time_slots):
        activity_start_time = add_minutes(start_time, index * time_resolution)
        activity = time_slots[index]["a"]
        comment = time_slots[index].get("c", "")
        room = time_slots[index].get("r", "")
        room_str = room_dict.get(room, "")
        activity_str = activityDict[activity]

        activity_count = 0
        comment_count = 0
        
        for i in range(index, len(time_slots)):
            if time_slots[i]["a"] == activity and time_slots[i].get("c", "") == comment:
                activity_count += 1
                comment_count += 1

                if activity_count == 30:
                    break   # Max for what PMO can handle
            else:
                break

        comment_minutes = comment_count * time_resolution

        if activity != "ns":
            pos = SetTimeSlots(activity_str, activity_start_time, activity_count, prev_count)
            prev_count = activity_count + 5
        else:
            pos = True
        
        if pos and comment:
            AddBooking(comment, activity_str, activity_start_time, comment_minutes)

        if pos and room:
            AddBooking(room_str, activity_str, activity_start_time, time_resolution)

        index += activity_count

        if not pos:
            break

    if activity != "ns":
        SetTimeSlots(activityDict["ns"], add_minutes(start_time, time_resolution * len(time_slots)), 6, prev_count)
    
    return pos


# pos = OpenPMO()
# if pos:
#     SetScheduleChunk(example_chunk)

In [45]:
def set_skips(continue_from_user, continue_from_date):
    skip_users = False
    skip_dates = False
    if continue_from_user:
        skip_users = True
    if continue_from_date:
        skip_dates = True

    return skip_users, skip_dates

# Actually run

In [46]:
continue_from_user = None # 8
continue_from_date = None # "2024-09-25"

In [47]:
OpenPMO()
pos = DeselectAllUsers()

total_time = 0
completed_users = 0
skip_users, skip_dates = set_skips(continue_from_user, continue_from_date)

if pos:
    for staff_schedule in filtered_staff_schedules:
        userId = staff_schedule["staffPk"]
        user = staff_dict[str(userId)]
        if skip_users:
            if userId == continue_from_user:
                skip_users = False
            else:
                print(f"Skipping {user}")
                continue

        print(f"""
###########################################
              # {user} #
###########################################""")
        
        start_time = time.time()
        
        DeselectSingleUser()
        pos = SetUser(user)

        if pos:
            for person_day in staff_schedule["personDays"]:
                date_str = person_day["date"]

                if skip_dates:
                    if date_str == continue_from_date:
                        skip_dates = False
                    else:
                        print(f"Skipping {date_str}")
                        continue

                print(date_str)
                pos = SetDate(date_str)
                if not pos:
                    break

                chunk = person_day["schedule_chunk"]
                # chunk_start_time = chunk["start_time"]
                # time_resolution = chunk["time_resolution"]
                # time_slots = chunk["time_slots"]
                
                pos = SetScheduleChunk(chunk)
                print("Completed day")
                if not pos:
                    break

        end_time = time.time()
        user_time = end_time - start_time
        total_time += user_time
        completed_users += 1

        print(f"\nCompleted {user} in {user_time / 60:.1f} minutes\n")

average_time = total_time / completed_users if completed_users > 0 else 0
print(f"Total time: {average_time / 60:.1f} minutes. Average time per user: {average_time / 60:.1f} minutes")


###########################################
              # Rasmus #
###########################################
Text: "und : Rasmus Arvidsson"
User correctly selected
User was selected correctly
2024-11-18
Completed day
2024-11-19
Completed day
2024-11-20
Completed day
2024-11-21
Completed day
2024-11-22
Completed day
2024-11-23
Completed day
2024-11-24
Completed day
2024-11-25
Completed day
2024-11-26
Completed day
2024-11-27
Completed day
2024-11-28
Completed day
2024-11-29
Completed day

Completed Rasmus in 14.5 minutes


###########################################
              # Linn #
###########################################
Text: "is Lund : Linn Widén"
User correctly selected
User was selected correctly
2024-11-18
Completed day
2024-11-19
Completed day
2024-11-20
Completed day
2024-11-21
Completed day
2024-11-22
Completed day
2024-11-23
Completed day
2024-11-24
Completed day
2024-11-25
Completed day
2024-11-26
Completed day
2024-11-27
Completed day
2024-11-28
Completed day


In [48]:
continue_from_user = None # 8
continue_from_date = None # "2024-09-25"

# Evaluate result

In [49]:
color_dict = {
    "ns": "#ffffff",  # Not scheduled
    "ad": "#c0c0c0",  # admin
    "em": "#ff0000",  # emergency
    "pl": "#008040",  # planned
    "ph": "#ffff80",  # phone
    "br": "#0080ff",  # break
    "me": "#804040",  # meeting
    "sp": "#80ff80",  # special
    "mi": "#fe4e12",  # miscellaneous
    "ch": "#ff80ff",  # childcare
    "tr": "#408080",  # team-rounds
    "wl": "#ff80c0",  # waitinglist
}

In [50]:
def get_color_bar_screenshot(pos, height=456):
    x = pos.x + 37
    y = pos.y - 55
    width = 6

    # Take the screenshot
    screenshot = pag.screenshot(region=(x-3, y, width, height))

    return screenshot

# Save the screenshot to a file
# screenshot.save('screenshot.png')

In [51]:
def get_colors_from_bar(bar: Image.Image):
    start_y = 8
    step = 19
    width, height = bar.size

    # Create an empty list to store the color samples
    pmo_colors = []

    # Get the color of each sample point and add it to the list
    for i in range(start_y, height, step):
        color = bar.getpixel((width // 2, i))
        pmo_colors.append(color)

    return pmo_colors

In [52]:
def scroll_to_bottom():
    pos = waitForElement(IMG.SixteentOClockMark, beforeStart=0.05, conf=0.98, limit=0.1, warn=False)
    if pos:
        return pos
    pos = waitAndClickElement(IMG.TenMinuteMark, relX=100)
    if pos:
        pos = False
        for i in range(16):
            if i == 0:
                pag.scroll(-2800)
            else:
                pag.scroll(-350)
            pos = waitForElement(IMG.SixteentOClockMark, beforeStart=0.05, conf=0.98, limit=0.1, warn=False)
            if pos:
                break
    return pos

def scroll_to_top():
    pos = waitForElement(IMG.EightOClockMark, beforeStart=0.05, conf=0.98, limit=0.1, warn=False)
    if pos:
        return pos
    pos = waitAndClickElement(IMG.TenMinuteMark, relX=100)
    if pos:
        pos = False
        for i in range(16):
            if i == 0:
                pag.scroll(2800)
            else:
                pag.scroll(350)
            pos = waitForElement(IMG.EightOClockMark, beforeStart=0.05, conf=0.98, limit=0.1, warn=False)
            if pos:
                break
        # pag.scroll(80)
    return pos

# OpenPMO()
# scroll_to_top()


In [53]:
def create_color_image(colors):
    # Create a new image with the specified size
    img = Image.new('RGB', (6, len(colors) * 19))

    # Create a draw object
    draw = ImageDraw.Draw(img)

    # Draw a rectangle for each color
    for i, color in enumerate(colors):
        top_left = (0, i * 19)
        bottom_right = (6, (i + 1) * 19)
        draw.rectangle([top_left, bottom_right], fill=color)

    # Return the image
    return img

In [54]:
def combine_images(list_a, list_b):
    # Calculate the total height and maximum width for each list
    y_height_a = sum(img.height for img in list_a)
    y_height_b = 0
    if (len(list_b) > 0):
        y_height_b = sum(img.height for img in list_b)
    y_total_max = max(y_height_a, y_height_b)
    x_max_a = max(img.width for img in list_a)
    x_max_b = 0
    if (len(list_b) > 0):
        x_max_b = max(img.width for img in list_b)
    x_total = x_max_a + x_max_b

    # Create a new image with the total width and height
    img = Image.new('RGB', (x_total, y_total_max))

    # Paste the images from list_a into the new image
    y = 0
    for image in list_a:
        img.paste(image, (0, y))
        y += image.height

    # Paste the images from list_b into the new image
    y = 0
    x = max(img.width for img in list_a)
    for image in list_b:
        img.paste(image, (x, y))
        y += image.height

    # Return the new image
    return img

In [55]:
# test_time_slots = staff_schedules[3]["personDays"][2]["schedule_chunk"]["time_slots"]
# test_time_slots

def hexa_to_pil_rgb(hexa):
    return tuple(int(hexa[i:i+2], 16) for i in (1, 3, 5))

def get_colors_from_time_slots(time_slots):
    color_list = []
    for time_slot in time_slots:
        try:
            activity = time_slot["a"]
            color = color_dict[activity]
            color_list.append(hexa_to_pil_rgb(color))
        except Exception:
            print(activity, color)
    return color_list

#chunk_to_color_list(test_time_slots)

In [56]:
def get_bar_image_and_colors():
    alreadyDidBottom = False
    pos = waitForElement(IMG.SixteentOClockMark, beforeStart=0.05, conf=0.98, limit=0.1, warn=False)
    if pos:
        pos = waitForElement(IMG.TwelveOClockMark)
        if pos:
            screenshot_from_twelve_thirty = get_color_bar_screenshot(pos, height=605)
            alreadyDidBottom = True
            scroll_to_top()
    
    pos = waitForElement(IMG.EightOClockMark)
    
    if pos:
        screenshot_from_eight = get_color_bar_screenshot(pos)
        if not alreadyDidBottom:
            scroll_to_bottom()

            pos = waitForElement(IMG.TwelveOClockMark)
            if pos:
                screenshot_from_twelve_thirty = get_color_bar_screenshot(pos, height=605)
        
    colors_morning = get_colors_from_bar(screenshot_from_eight)
    colors_afternoon = get_colors_from_bar(screenshot_from_twelve_thirty)
    all_colors = colors_morning + colors_afternoon
    combined_image = combine_images([screenshot_from_eight, screenshot_from_twelve_thirty], [])

    return combined_image, all_colors

# OpenPMO()
# combined_image, all_colors = get_bar_image_and_colors()
# combined_image.show()

# Actually evaluate

In [57]:
# pos = OpenPMO()
# pos = DeselectAllUsers()

total_time = 0
completed_users = 0
collected_bar_images = {}
extracted_colors = {}
time_slot_colors = {}
skip_users, skip_dates = set_skips(continue_from_user, continue_from_date)

if pos:
    for staff_schedule in filtered_staff_schedules:
        userId = staff_schedule["staffPk"]
        user = staff_dict[str(userId)]
        if skip_users:
            if userId == continue_from_user:
                skip_users = False
            else:
                print(f"Skipping {user}")
                continue

        print(f"""
###########################################
              # {user} #
###########################################""")
        
        start_time = time.time()
        
        DeselectSingleUser()
        pos = SetUser(user)

        if pos:
            pos = waitForElement(IMG.IdagButton)
            if pos:
                userImage = pag.screenshot(region=(pos.x, pos.y + 13, 500, 20 ))
                userImage.save(f"ValidationImages/{user}_user.png")

        if pos:
            for person_day in staff_schedule["personDays"]:
                date_str = person_day["date"]

                if skip_dates:
                    if date_str == continue_from_date:
                        skip_dates = False
                    else:
                        print(f"Skipping {date_str}")
                        continue

                print(date_str)
                pos = SetDate(date_str)
                if not pos:
                    break

                image, colors = get_bar_image_and_colors()
                collected_bar_images[f"{user}_{date_str}"] = image
                extracted_colors[f"{user}_{date_str}"] = colors
                time_slot_colors[f"{user}_{date_str}"] = get_colors_from_time_slots(person_day["schedule_chunk"]["time_slots"])

                print("Completed day")
                if not pos:
                    break

        end_time = time.time()
        user_time = end_time - start_time
        total_time += user_time
        completed_users += 1

        print(f"\nCompleted {user} in {user_time / 60:.1f} minutes\n")

average_time = total_time / completed_users if completed_users > 0 else 0
print(f"Total time: {total_time / 60:.1f} minutes. Average time per user: {average_time / 60:.1f} minutes")


###########################################
              # Rasmus #
###########################################
Text: "und : Rasmus Arvidsson"
User correctly selected
User was selected correctly
2024-11-18
Completed day
2024-11-19
Completed day
2024-11-20
Completed day
2024-11-21
Completed day
2024-11-22
Completed day
2024-11-23
Completed day
2024-11-24
Completed day
2024-11-25
Completed day
2024-11-26
Completed day
2024-11-27
Completed day
2024-11-28
Completed day
2024-11-29
Completed day

Completed Rasmus in 1.8 minutes


###########################################
              # Linn #
###########################################
Text: "is Lund : Linn Widén"
User correctly selected
User was selected correctly
2024-11-18
Completed day
2024-11-19
Completed day
2024-11-20
Completed day
2024-11-21
Completed day
2024-11-22
Completed day
2024-11-23
Completed day
2024-11-24
Completed day
2024-11-25
Completed day
2024-11-26
Completed day
2024-11-27
Completed day
2024-11-28
Completed day
2

# Combine images and evaluate

In [58]:
discrepancyDict = {}
skip_users, skip_dates = set_skips(continue_from_user, continue_from_date)

for staff_schedule in filtered_staff_schedules:
    userId = staff_schedule["staffPk"]
    user = staff_dict[str(userId)]
    if skip_users:
        if userId == continue_from_user:
            skip_users = False
        else:
            print(f"Skipping {user}")
            continue

    print(f"""
###########################################
            # {user} #
###########################################""")

    for person_day in staff_schedule["personDays"]:
        date_str = person_day["date"]

        if skip_dates:
            if date_str == continue_from_date:
                skip_dates = False
            else:
                print(f"Skipping {date_str}")
                continue
        print(date_str)

        key = f"{user}_{date_str}"
        extracted_image = collected_bar_images[key]
        extracted_color_list = extracted_colors[key]
        time_slot_color_list = time_slot_colors[key]
        time_slot_color_image = create_color_image(time_slot_color_list)
        combined_image = combine_images([extracted_image], [time_slot_color_image])
        combined_image.save(f"ValidationImages/{user}_{date_str}.png")

        discrepancies = 0

        for i in range(len(time_slot_color_list)):
            ref_color = time_slot_color_list[i]
            pmo_color = extracted_color_list[i]
            if ref_color != pmo_color:
                discrepancies += 1
            # print(f"Comparing {ref_color} with {pmo_color} => {discrepancies} discrepancies")

        discrepancyDict[key] = discrepancies
            
print(f"\nCompleted")


###########################################
            # Rasmus #
###########################################
2024-11-18
2024-11-19
2024-11-20
2024-11-21
2024-11-22
2024-11-23
2024-11-24
2024-11-25
2024-11-26
2024-11-27
2024-11-28
2024-11-29

###########################################
            # Linn #
###########################################
2024-11-18
2024-11-19
2024-11-20
2024-11-21
2024-11-22
2024-11-23
2024-11-24
2024-11-25
2024-11-26
2024-11-27
2024-11-28
2024-11-29

###########################################
            # Maria MC #
###########################################
2024-11-18
2024-11-19
2024-11-20
2024-11-21
2024-11-22
2024-11-23
2024-11-24
2024-11-25
2024-11-26
2024-11-27
2024-11-28
2024-11-29

###########################################
            # Maria K #
###########################################
2024-11-18
2024-11-19
2024-11-20
2024-11-21
2024-11-22
2024-11-23
2024-11-24
2024-11-25
2024-11-26
2024-11-27
2024-11-28
2024-11-29

######################

# Generate html-doc

In [59]:
validation_html = """
<!DOCTYPE html>
<html>
<head>
    <style>
    </style>
</head>
<body>

"""

skip_users, skip_dates = set_skips(continue_from_user, continue_from_date)
for staff_schedule in filtered_staff_schedules:
        userId = staff_schedule["staffPk"]
        user = staff_dict[str(userId)]
        if skip_users:
            if userId == continue_from_user:
                skip_users = False
            else:
                validation_html += f'<h2>{user}</h2>\n<p>Skipped</p>\n'
                print(f"Skipping {user}")
                continue

        print(f"""
###########################################
              # {user} #
###########################################""")
        validation_html += f'<h2>{user}</h2>\n'
        validation_html += f'<img src="{user}_user.png"/>\n'

        skipped_dates = []
        used_dates = []
        used_images = []

        for person_day in staff_schedule["personDays"]:
            date_str = person_day["date"]

            if skip_dates:
                if date_str == continue_from_date:
                    skip_dates = False
                else:
                    print(f"Skipping {date_str}")
                    skipped_dates.append(date_str)
                    continue

            used_dates.append(date_str)
            used_images.append(f'<img src="{user}_{date_str}.png"/>')
            print(date_str)
            
        validation_html += "<table>\n<tr>\n"
        bg_colors = []
        for date in skipped_dates:
             validation_html += f'<th style="background-color:#cccccc;">{date}</th>\n'
        for date in used_dates:
            hasDiscrepancies = discrepancyDict.get(f"{user}_{date}", 0) > 0
            bg_color = "#ccffcc;" if not hasDiscrepancies else "#ffcccc"
            bg_colors.append(bg_color)
            validation_html += f'<th style="background-color:{bg_color};">{date}</th>\n'
        validation_html += "</tr>\n<tr>\n"
        for data in skipped_dates:
            validation_html += f'<td class="grey">-</td>\n'
        for i in range(len(used_images)):
            image = used_images[i]
            bg_color = bg_colors[i]
            validation_html += f'<td style="background-color:{bg_color};">{image}</td>\n'
        validation_html += "</tr>\n</table>\n"

        print(f"\nCompleted")
    
validation_html += """
</body>
</html>
"""

with open("ValidationImages/validation.html", "w") as file:
    file.write(validation_html)


###########################################
              # Rasmus #
###########################################
2024-11-18
2024-11-19
2024-11-20
2024-11-21
2024-11-22
2024-11-23
2024-11-24
2024-11-25
2024-11-26
2024-11-27
2024-11-28
2024-11-29

Completed

###########################################
              # Linn #
###########################################
2024-11-18
2024-11-19
2024-11-20
2024-11-21
2024-11-22
2024-11-23
2024-11-24
2024-11-25
2024-11-26
2024-11-27
2024-11-28
2024-11-29

Completed

###########################################
              # Maria MC #
###########################################
2024-11-18
2024-11-19
2024-11-20
2024-11-21
2024-11-22
2024-11-23
2024-11-24
2024-11-25
2024-11-26
2024-11-27
2024-11-28
2024-11-29

Completed

###########################################
              # Maria K #
###########################################
2024-11-18
2024-11-19
2024-11-20
2024-11-21
2024-11-22
2024-11-23
2024-11-24
2024-11-25
2024-11-26
2024-11-27
2024